In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import robust_scale
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import scipy.stats as stats
import statistics

In [2]:
data = pd.read_csv('CloudFactory_DataSet_Accuracy_Prediction.tsv', sep ='\t')
data.head()

,timestamp,task_id,worker_id,keypress_keytypeint,mousemove_distance,mouseclick_button,work_duration,accuracy
0,1470267718900,57a28146e2013f2463dcd0b8,512dbf09c32206698e000518,4.0,NaN,NaN,23.884,1.0
1,1470267724803,57a28146e2013f2463dcd0b8,512dbf09c32206698e000518,6.0,NaN,NaN,23.884,1.0
2,1470267724823,57a28146e2013f2463dcd0b8,512dbf09c32206698e000518,6.0,NaN,NaN,23.884,1.0
3,1470267725117,57a28146e2013f2463dcd0b8,512dbf09c32206698e000518,12.0,NaN,NaN,23.884,1.0
4,1470267725276,57a28146e2013f2463dcd0b8,512dbf09c32206698e000518,12.0,NaN,NaN,23.884,1.0


In [3]:
data = data.assign(keytype_changed= "0")

In [4]:
data = pd.read_csv('master.csv')

In [5]:
data.head()

,accuracy,keypress_keytypeint,keytype_changed,mouseclick_button,mousemove_distance,task_id,timestamp,work_duration,worker_id
0,1.0,4,0,0,0.0,57a28146e2013f2463dcd0b8,1.470270e+12,23.884,512dbf09c32206698e000518
1,1.0,6,1,0,0.0,57a28146e2013f2463dcd0b8,1.470270e+12,23.884,512dbf09c32206698e000518
2,1.0,6,0,0,0.0,57a28146e2013f2463dcd0b8,1.470270e+12,23.884,512dbf09c32206698e000518
3,1.0,12,1,0,0.0,57a28146e2013f2463dcd0b8,1.470270e+12,23.884,512dbf09c32206698e000518
4,1.0,12,0,0,0.0,57a28146e2013f2463dcd0b8,1.470270e+12,23.884,512dbf09c32206698e000518


In [6]:
data = data.groupby(['worker_id','task_id']).mean()

In [7]:
data = data.reset_index(level=0)

In [8]:
data = data.reset_index(level = 0)

In [9]:
data.head()

,task_id,worker_id,accuracy,keypress_keytypeint,keytype_changed,mouseclick_button,mousemove_distance,timestamp,work_duration
0,57a2997ee2013f0c5a5d9f33,501767d1cb022453170004dc,0.833333,6.879310,0.517241,0.068966,19.722592,1.470270e+12,88.318
1,57a2997ee2013f0c5a5d9f34,501767d1cb022453170004dc,1.000000,3.463768,0.217391,0.376812,7.173393,1.470270e+12,34.392
2,57a2997ee2013f0c5a5d9f35,501767d1cb022453170004dc,1.000000,4.918367,0.346939,0.061224,22.958495,1.470270e+12,18.640
3,57a2997ee2013f0c5a5d9f36,501767d1cb022453170004dc,0.833333,6.209302,0.372093,0.093023,8.415718,1.470270e+12,15.841
4,57a2997ee2013f0c5a5d9f37,501767d1cb022453170004dc,1.000000,5.755556,0.333333,0.155556,2.868044,1.470270e+12,15.844


In [10]:
del data['timestamp']

In [11]:
data.head()

,task_id,worker_id,accuracy,keypress_keytypeint,keytype_changed,mouseclick_button,mousemove_distance,work_duration
0,57a2997ee2013f0c5a5d9f33,501767d1cb022453170004dc,0.833333,6.879310,0.517241,0.068966,19.722592,88.318
1,57a2997ee2013f0c5a5d9f34,501767d1cb022453170004dc,1.000000,3.463768,0.217391,0.376812,7.173393,34.392
2,57a2997ee2013f0c5a5d9f35,501767d1cb022453170004dc,1.000000,4.918367,0.346939,0.061224,22.958495,18.640
3,57a2997ee2013f0c5a5d9f36,501767d1cb022453170004dc,0.833333,6.209302,0.372093,0.093023,8.415718,15.841
4,57a2997ee2013f0c5a5d9f37,501767d1cb022453170004dc,1.000000,5.755556,0.333333,0.155556,2.868044,15.844


In [12]:
data['keytype_changed'].corr(data['accuracy'])

-0.026644051093336132

In [13]:
data['accuracy'] = data['accuracy'] * 6
data['accuracy'] = data['accuracy'].round()

In [14]:
data.loc[data['accuracy']!=6, 'accuracy'] = 0

In [15]:
data['accuracy'].value_counts()

6.0    2626
0.0    2374
Name: accuracy, dtype: int64

In [16]:
data['keytype_changed'].corr(data['accuracy'])

0.010084178764958691

In [17]:
x = data.drop(['task_id','worker_id','keypress_keytypeint','accuracy'], axis =1)
y = data['accuracy']

x_train, x_test, y_train, y_test = train_test_split (x,y, test_size = 0.35, random_state = 101)

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
logmod = LogisticRegression()

In [20]:
logmod.fit( x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
predictions = logmod.predict(x_test)

In [22]:
from sklearn.metrics import classification_report
print (classification_report(y_test,predictions))

             precision    recall  f1-score   support

        0.0       0.63      0.32      0.43       857
        6.0       0.56      0.82      0.66       893

avg / total       0.59      0.58      0.55      1750



In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions,)

array([[277, 580],
       [162, 731]])